In [393]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [394]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [395]:
tf.keras.backend.clear_session()

In [396]:
def add_cos_monthly(df, column_name):
    values = [np.cos(2 * np.pi * (i % 12) / 12) for i in range(len(df))]
    df[column_name] = values
    return df

In [397]:
subject = 'Amazonas - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [398]:
data = pd.read_csv('2003_mo_model_input_AM.csv')
# data = add_cos_monthly(data, "Sazonalidade")
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Consumo de Cimento (t)
0,2003-1,0.695898,0.330279,8.630942,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,51.313080,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,28.193
1,2003-2,0.696043,0.331137,8.624872,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,51.554195,4.760190e+07,2.521321e+06,12.492164,4.320154e+07,31.226
2,2003-3,0.696187,0.331306,8.618803,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,51.523306,4.765911e+07,2.522846e+06,12.493707,4.321789e+07,33.484
3,2003-4,0.696332,0.331791,8.612734,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,51.639628,4.771632e+07,2.524370e+06,12.495250,4.323425e+07,31.596
4,2003-5,0.696476,0.332189,8.606665,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,51.838456,4.777353e+07,2.525895e+06,12.496793,4.325060e+07,32.919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.746275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.352061,NaN,NaN,NaN,NaN,66.077
236,2022-9,NaN,0.746582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.160630,NaN,NaN,NaN,NaN,58.590
237,2022-10,NaN,0.745406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.937187,NaN,NaN,NaN,NaN,58.793
238,2022-11,NaN,0.742703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.889598,NaN,NaN,NaN,NaN,53.509


In [399]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
0,0.000000,0.000000,0.052061,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.577216,0.000000,0.613647,0.879074,0.000000
1,0.008340,0.002941,0.050819,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.585070,0.006887,0.624331,0.887854,0.014567
2,0.016679,0.003520,0.049577,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.584064,0.013774,0.635015,0.896634,0.029134
3,0.025019,0.005180,0.048335,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.587853,0.020662,0.645699,0.905414,0.043701
4,0.033358,0.006545,0.047093,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.594330,0.027549,0.656383,0.914194,0.058268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.903741,0.754487,0.866685,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.088348,0.982205,0.133165,0.277328,0.991832
188,0.901065,0.766802,0.865216,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.100096,0.979663,0.143224,0.280010,0.990665
189,0.898389,0.781833,0.863748,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.110341,0.977121,0.153284,0.282691,0.989498
190,0.895714,0.797554,0.862280,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.121334,0.974579,0.163343,0.285373,0.988332


In [400]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      34.293
1      48.623
2      34.555
3      57.805
4      39.036
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amazonas - Consumo de Cimento (t), Length: 240, dtype: float64

In [401]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
0,0.000000,0.000000,0.052061,1.000000,1.000000,0.746035,1.000000,0.000000,0.000000,1.000000,0.577216,0.000000,0.613647,0.879074,0.000000
1,0.008340,0.002941,0.050819,0.921240,0.965719,0.580259,0.936603,0.007992,0.011007,0.968938,0.585070,0.006887,0.624331,0.887854,0.014567
2,0.016679,0.003520,0.049577,0.873107,0.927412,0.926926,0.906543,0.015984,0.022015,0.948438,0.584064,0.013774,0.635015,0.896634,0.029134
3,0.025019,0.005180,0.048335,0.851659,0.885588,0.549924,0.858675,0.023976,0.033022,0.928794,0.587853,0.020662,0.645699,0.905414,0.043701
4,0.033358,0.006545,0.047093,0.823277,0.838034,0.807277,0.824839,0.031968,0.044030,0.970421,0.594330,0.027549,0.656383,0.914194,0.058268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.990194,0.701866,0.992455,0.379453,0.024030,0.338208,0.039971,0.890665,0.956993,0.182507,0.422086,0.974040,0.028603,0.025547,0.880428
158,0.991086,0.687106,0.984909,0.332832,0.021531,0.371581,0.027548,0.893949,0.953662,0.168812,0.399922,0.975961,0.026003,0.034675,0.886450
159,0.991977,0.675264,0.977364,0.313954,0.019385,0.365349,0.037038,0.897232,0.950331,0.152959,0.378842,0.977883,0.023403,0.043802,0.892473
160,0.992869,0.660150,0.969819,0.294856,0.017671,0.404993,0.070841,0.900516,0.947000,0.137613,0.359110,0.979804,0.020802,0.052929,0.898495


In [402]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      34.293
1      48.623
2      34.555
3      57.805
4      39.036
        ...  
157    44.068
158    62.847
159    30.374
160    60.003
161    81.014
Name: Amazonas - Consumo de Cimento (t), Length: 162, dtype: float64

In [403]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [404]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [405]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
126,0.734848,0.953402,0.804791,0.612440,0.052718,0.094523,0.201912,0.780557,0.994213,0.372054,0.795202,0.850220,0.403398,0.230253,0.851684
127,0.737399,0.962015,0.815045,0.626227,0.056653,0.200555,0.209545,0.784414,0.995177,0.375438,0.778918,0.855511,0.383186,0.213663,0.849733
128,0.739950,0.967720,0.825298,0.645442,0.060771,0.214993,0.238038,0.788271,0.996142,0.372254,0.751913,0.860801,0.362974,0.197073,0.847782
129,0.742501,0.973113,0.835551,0.673151,0.065105,0.053026,0.249843,0.792128,0.997106,0.369458,0.739202,0.866092,0.342761,0.180482,0.845831
130,0.745052,0.977467,0.845804,0.699949,0.069099,0.016760,0.266770,0.795984,0.998071,0.371547,0.722044,0.871383,0.322549,0.163892,0.843880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.903741,0.754487,0.866685,0.000000,0.030808,0.063780,0.381401,0.987600,0.828698,0.005731,0.088348,0.982205,0.133165,0.277328,0.991832
188,0.901065,0.766802,0.865216,0.029504,0.027720,0.048101,0.466609,0.990700,0.823090,0.037761,0.100096,0.979663,0.143224,0.280010,0.990665
189,0.898389,0.781833,0.863748,0.043125,0.023074,0.016800,0.530936,0.993800,0.817482,0.087044,0.110341,0.977121,0.153284,0.282691,0.989498
190,0.895714,0.797554,0.862280,0.059756,0.017547,0.035186,0.611274,0.996900,0.811875,0.118264,0.121334,0.974579,0.163343,0.285373,0.988332


In [406]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [407]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [408]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 10)
#     target,target_val = validation_splitter(train_target, 10)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(144, activation='tanh', 
                             return_sequences=True,
                             kernel_regularizer=tf.keras.regularizers.l1(0.01),
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True,
                             kernel_regularizer=tf.keras.regularizers.l1(0.01)),
        tf.keras.layers.LSTM(18, activation='tanh', 
                             return_sequences=False),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [409]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [410]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2116090008, 1100089401, 1935576758, 3800130212, 2975958399, 2843256582, 2957945071, 1927493109, 597265619, 4195686783]


Step: 0 ___________________________________________
val_loss: 122.85660552978516
winner_seed: 2116090008


Step: 1 ___________________________________________
val_loss: 122.85337829589844
winner_seed: 1100089401


Step: 2 ___________________________________________
val_loss: 122.8597412109375


Step: 3 ___________________________________________
val_loss: 122.85664367675781


Step: 4 ___________________________________________
val_loss: 122.85365295410156


Step: 5 ___________________________________________
val_loss: 122.85438537597656


Step: 6 ___________________________________________
val_loss: 122.86080932617188


Step: 7 ___________________________________________
val_loss: 122.85211944580078
winner_seed: 1927493109


Step: 8 ___________________________________________
val_loss: 122.85598754882812


Step: 9 ___________________________________________
val_loss

In [411]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 396ms/step - loss: 5017.6719 - val_loss: 2914.8005
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 4820.6904 - val_loss: 2814.3311
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 4702.2700 - val_loss: 2743.8247
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 4617.3232 - val_loss: 2689.7092
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 4549.5757 - val_loss: 2643.8840
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 4492.6460 - val_loss: 2605.0378
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 4444.9849 - val_loss: 2572.1338
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 4404.1558 - val_loss: 2545.5027
Epoch 9/10000
4/4 [==============================] - 0s 10ms/step - loss: 4371.0747 - val_loss: 2523.3403
Epoch 10/10000
4/4 [==============================]

4/4 [==============================] - 0s 9ms/step - loss: 3613.0618 - val_loss: 1963.5913
Epoch 79/10000
4/4 [==============================] - 0s 10ms/step - loss: 3604.2085 - val_loss: 1957.0354
Epoch 80/10000
4/4 [==============================] - 0s 10ms/step - loss: 3595.2959 - val_loss: 1950.5155
Epoch 81/10000
4/4 [==============================] - 0s 10ms/step - loss: 3586.3892 - val_loss: 1944.0330
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 3577.6311 - val_loss: 1937.5513
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 3568.8684 - val_loss: 1931.0717
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 3560.0796 - val_loss: 1924.6169
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 3551.2688 - val_loss: 1918.1881
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 3542.5437 - val_loss: 1911.7604
Epoch 87/10000
4/4 [==============================] - 0s

4/4 [==============================] - 0s 9ms/step - loss: 2992.8210 - val_loss: 1515.5972
Epoch 155/10000
4/4 [==============================] - 0s 9ms/step - loss: 2985.2876 - val_loss: 1510.3019
Epoch 156/10000
4/4 [==============================] - 0s 9ms/step - loss: 2977.8325 - val_loss: 1505.0100
Epoch 157/10000
4/4 [==============================] - 0s 9ms/step - loss: 2970.2998 - val_loss: 1499.7462
Epoch 158/10000
4/4 [==============================] - 0s 11ms/step - loss: 2962.8716 - val_loss: 1494.4816
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 2955.4534 - val_loss: 1489.2235
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 2947.9978 - val_loss: 1483.9761
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 2940.6001 - val_loss: 1478.7448
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 2933.0745 - val_loss: 1473.5442
Epoch 163/10000
4/4 [==============================

4/4 [==============================] - 0s 10ms/step - loss: 2464.8245 - val_loss: 1150.8002
Epoch 231/10000
4/4 [==============================] - 0s 10ms/step - loss: 2458.4751 - val_loss: 1146.5060
Epoch 232/10000
4/4 [==============================] - 0s 9ms/step - loss: 2452.0813 - val_loss: 1142.2273
Epoch 233/10000
4/4 [==============================] - 0s 9ms/step - loss: 2445.8545 - val_loss: 1137.9263
Epoch 234/10000
4/4 [==============================] - 0s 9ms/step - loss: 2439.3745 - val_loss: 1133.6808
Epoch 235/10000
4/4 [==============================] - 0s 9ms/step - loss: 2433.1335 - val_loss: 1129.4237
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 2426.7109 - val_loss: 1125.2063
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 2420.4871 - val_loss: 1120.9604
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 2414.1216 - val_loss: 1116.7358
Epoch 239/10000
4/4 [==============================

4/4 [==============================] - 0s 9ms/step - loss: 2015.2219 - val_loss: 856.0553
Epoch 307/10000
4/4 [==============================] - 0s 9ms/step - loss: 2009.7472 - val_loss: 852.6115
Epoch 308/10000
4/4 [==============================] - 0s 10ms/step - loss: 2004.3658 - val_loss: 849.1713
Epoch 309/10000
4/4 [==============================] - 0s 10ms/step - loss: 1998.8723 - val_loss: 845.7609
Epoch 310/10000
4/4 [==============================] - 0s 9ms/step - loss: 1993.5118 - val_loss: 842.3417
Epoch 311/10000
4/4 [==============================] - 0s 10ms/step - loss: 1988.0964 - val_loss: 838.9304
Epoch 312/10000
4/4 [==============================] - 0s 9ms/step - loss: 1982.7573 - val_loss: 835.5040
Epoch 313/10000
4/4 [==============================] - 0s 9ms/step - loss: 1977.3571 - val_loss: 832.0912
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 1971.9873 - val_loss: 828.6988
Epoch 315/10000
4/4 [==============================] - 0s 9

...

Epoch 975/10000
4/4 [==============================] - 0s 10ms/step - loss: 288.0607 - val_loss: 158.7025
Epoch 976/10000
4/4 [==============================] - 0s 9ms/step - loss: 287.3846 - val_loss: 159.0876
Epoch 977/10000
4/4 [==============================] - 0s 9ms/step - loss: 286.7785 - val_loss: 159.4843
Epoch 978/10000
4/4 [==============================] - 0s 9ms/step - loss: 286.1477 - val_loss: 159.8838
Epoch 979/10000
4/4 [==============================] - 0s 9ms/step - loss: 285.5100 - val_loss: 160.2848
Epoch 980/10000
4/4 [==============================] - 0s 9ms/step - loss: 284.9137 - val_loss: 160.6969
Epoch 981/10000
4/4 [==============================] - 0s 9ms/step - loss: 284.2889 - val_loss: 161.1113
Epoch 982/10000
4/4 [==============================] - 0s 9ms/step - loss: 283.6641 - val_loss: 161.5227
Epoch 983/10000
4/4 [==============================] - 0s 9ms/step - loss: 283.0307 - val_loss: 161.9340
Epoch 984/10000
4/4 [==============================] -

4/4 [==============================] - 0s 9ms/step - loss: 248.5339 - val_loss: 192.3517
Epoch 1053/10000
4/4 [==============================] - 0s 9ms/step - loss: 248.1493 - val_loss: 192.8330
Epoch 1054/10000
4/4 [==============================] - 0s 9ms/step - loss: 247.7323 - val_loss: 193.2978
Epoch 1055/10000
4/4 [==============================] - 0s 9ms/step - loss: 247.3489 - val_loss: 193.7588
Epoch 1056/10000
4/4 [==============================] - 0s 9ms/step - loss: 246.9805 - val_loss: 194.2219
Epoch 1057/10000
4/4 [==============================] - 0s 9ms/step - loss: 246.5765 - val_loss: 194.6751
Epoch 1058/10000
4/4 [==============================] - 0s 9ms/step - loss: 246.1822 - val_loss: 195.1280
Epoch 1059/10000
4/4 [==============================] - 0s 10ms/step - loss: 245.7997 - val_loss: 195.5787
Epoch 1060/10000
4/4 [==============================] - 0s 9ms/step - loss: 245.4163 - val_loss: 196.0310
Epoch 1061/10000
4/4 [==============================] - 0s 9ms

4/4 [==============================] - 0s 10ms/step - loss: 223.8050 - val_loss: 229.3938
Epoch 1130/10000
4/4 [==============================] - 0s 11ms/step - loss: 223.5833 - val_loss: 229.9048
Epoch 1131/10000
4/4 [==============================] - 0s 11ms/step - loss: 223.2901 - val_loss: 230.3661
Epoch 1132/10000
4/4 [==============================] - 0s 11ms/step - loss: 223.0721 - val_loss: 230.8405
Epoch 1133/10000
4/4 [==============================] - 0s 10ms/step - loss: 222.8623 - val_loss: 231.3297
Epoch 1134/10000
4/4 [==============================] - 0s 10ms/step - loss: 222.5911 - val_loss: 231.7811
Epoch 1135/10000
4/4 [==============================] - 0s 10ms/step - loss: 222.3762 - val_loss: 232.2500
Epoch 1136/10000
4/4 [==============================] - 0s 10ms/step - loss: 222.1452 - val_loss: 232.7256
Epoch 1137/10000
4/4 [==============================] - 0s 10ms/step - loss: 221.9220 - val_loss: 233.1946
Epoch 1138/10000
4/4 [==============================] 

4/4 [==============================] - 0s 9ms/step - loss: 209.5940 - val_loss: 265.0284
Epoch 1207/10000
4/4 [==============================] - 0s 9ms/step - loss: 209.4486 - val_loss: 265.4558
Epoch 1208/10000
4/4 [==============================] - 0s 9ms/step - loss: 209.3447 - val_loss: 265.9266
Epoch 1209/10000
4/4 [==============================] - 0s 9ms/step - loss: 209.1863 - val_loss: 266.3555
Epoch 1210/10000
4/4 [==============================] - 0s 9ms/step - loss: 209.0675 - val_loss: 266.8028
Epoch 1211/10000
4/4 [==============================] - 0s 9ms/step - loss: 208.9181 - val_loss: 267.2202
Epoch 1212/10000
4/4 [==============================] - 0s 9ms/step - loss: 208.8122 - val_loss: 267.6735
Epoch 1213/10000
4/4 [==============================] - 0s 9ms/step - loss: 208.6781 - val_loss: 268.1134
Epoch 1214/10000
4/4 [==============================] - 0s 9ms/step - loss: 208.5465 - val_loss: 268.5495
Epoch 1215/10000
4/4 [==============================] - 0s 9ms/

4/4 [==============================] - 0s 10ms/step - loss: 201.9639 - val_loss: 297.0904
Epoch 1284/10000
4/4 [==============================] - 0s 10ms/step - loss: 201.8950 - val_loss: 297.4129
Epoch 1285/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.8382 - val_loss: 297.7380
Epoch 1286/10000
4/4 [==============================] - 0s 10ms/step - loss: 201.7741 - val_loss: 298.0640
Epoch 1287/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.7143 - val_loss: 298.4119
Epoch 1288/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.6615 - val_loss: 298.7975
Epoch 1289/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.5903 - val_loss: 299.1524
Epoch 1290/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.5287 - val_loss: 299.4967
Epoch 1291/10000
4/4 [==============================] - 0s 9ms/step - loss: 201.4806 - val_loss: 299.8806
Epoch 1292/10000
4/4 [==============================] - 0s 9

In [412]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [413]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814,54.241814
Target,64.862,49.885,53.13,57.796,47.026,74.919,34.733,15.022,20.879,36.409,22.277,20.529,37.061,23.387,38.399,39.652,40.204,21.625,42.731,36.026,34.609,40.787,39.823,39.957,52.048,53.567,46.239,51.066,51.684,44.907
Error,10.620186,4.356815,1.111813,3.554188,7.215813,20.677185,19.508812,39.219814,33.362816,17.832813,31.964813,33.712814,17.180813,30.854815,15.842815,14.589813,14.037815,32.616814,11.510815,18.215813,19.632812,13.454815,14.418812,14.284813,2.193813,0.674812,8.002815,3.175812,2.557816,9.334812


In [414]:
display(mae)
display(mape)

15.52395

0.548674

In [415]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [416]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[650.9018]] - Target[497.4669999999999]| =  Error: [[153.43478]]; MAPE:[[0.30843207]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[650.9018]] - Target[434.26099999999997]| =  Error: [[216.64081]]; MAPE:[[0.49887237]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[325.4509]] - Target[299.511]| =  Error: [[25.93991]]; MAPE:[[0.08660755]]


[array([[153.43478]], dtype=float32),
 array([[216.64081]], dtype=float32),
 array([[25.93991]], dtype=float32)]

132.00517

0.29797065